In [1]:
import nltk 
import pandas as pd
import numpy as np

In [2]:

from google.colab import drive 
drive.mount('/content/gdrive')
# '/content/gdrive/MyDrive/Colab Notebooks/NLP/week6/SMSSpamCollection.txt'
df=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/NLP/week7/airline_review_training_data.csv')
# df = pd.read_csv("./airline_review_training_data.csv")
df

Mounted at /content/gdrive


,text,airline_sentiment
0,@USAirways BF has been stuck in CLT all day. I...,0
1,@united DM sent. This lack if customer servic...,0
2,@JetBlue thank you for the information.,1
3,@AmericanAir I DMed you my AA &amp; phone #s &...,0
4,@united @44Stocker my wife Sarah stocker did a...,0
...,...,...
8073,@JetBlue Apparently the pilot had made some an...,0
8074,@united customer service sucks! They hang up ...,0
8075,@USAirways I paid for my seat. I expect to be...,0
8076,"@united so, not only were you Late Flight, you...",0


In [14]:
# Cleaning Training Data set

import re
training_data_clean = (
    # start with training data
    df
    # remove all the '\n' values with space
     .assign(clean_document= lambda x: [re.sub(r"&amp;","",text) for text in x.text])
    #Remove “n\”
    .assign(clean_document= lambda x: [re.sub(r"\n","",text) for text in x.clean_document] )
    #Remove mentions i.e. any alphanumeric starting with “@”
    .assign(clean_document= lambda x: [re.sub("@[A-Za-z0-9_]+","",text) for text in x.clean_document])
    #Remove all hashtags i.e. “#”
    .assign(clean_document= lambda x: [re.sub("#[A-Za-z0-9_]+","",text) for text in x.clean_document])
#     
#     .assign(clean_document= lambda x: [re.sub(r"#","",text) for text in x.clean_document])
#    Remove all links i.e. any alphanumeric starting with https or http
    .assign(clean_document= lambda x: [re.sub(r"http\S+","",text) for text in x.clean_document])   
)
training_data_clean



,text,airline_sentiment,clean_document
0,@USAirways BF has been stuck in CLT all day. I...,0,BF has been stuck in CLT all day. Is the loun...
1,@united DM sent. This lack if customer servic...,0,DM sent. This lack if customer service is ge...
2,@JetBlue thank you for the information.,1,thank you for the information.
3,@AmericanAir I DMed you my AA &amp; phone #s &...,0,I DMed you my AA phone you can't have some...
4,@united @44Stocker my wife Sarah stocker did a...,0,my wife Sarah stocker did also called but co...
...,...,...,...
8073,@JetBlue Apparently the pilot had made some an...,0,Apparently the pilot had made some announceme...
8074,@united customer service sucks! They hang up ...,0,customer service sucks! They hang up after w...
8075,@USAirways I paid for my seat. I expect to be...,0,I paid for my seat. I expect to be able to u...
8076,"@united so, not only were you Late Flight, you...",0,"so, not only were you Late Flight, you broke ..."


In [15]:
# Feature Extraction
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 100)
# fit_transform(raw_documents[, y])===>Learn vocabulary and idf, return document-term matrix.
X = vectorizer.fit_transform([i for i in training_data_clean['clean_document']])
df_tfid_sklearn = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())
Y_train = training_data_clean['airline_sentiment']
X_train =df_tfid_sklearn.iloc[:, :-1] # aLL ROWS EXCEPT LAST
Y_train


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


0       0
1       0
2       1
3       0
4       0
       ..
8073    0
8074    0
8075    0
8076    0
8077    1
Name: airline_sentiment, Length: 8078, dtype: int64

In [16]:
# size of positive and negative sentiments
print('Positive Sentiments ', round(training_data_clean['airline_sentiment'].value_counts()[1]/len(training_data_clean) * 100,2), '% of the dataset')
print('Negative Sentimens', round(training_data_clean['airline_sentiment'].value_counts()[0]/len(training_data_clean) * 100,2), '% of the dataset')

Positive Sentiments  20.48 % of the dataset
Negative Sentimens 79.52 % of the dataset


In [25]:
# Training the classifier & predicting on test data
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

classifier = MultinomialNB()
# Learn using fit method
classifier.fit(X_train, Y_train)

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, Y_train)

clf_SVM = svm.SVC()
clf_SVM.fit(X_train, Y_train)

SVC()

In [26]:
# Cleaning the test Data
# /content/gdrive/MyDrive/Colab Notebooks/NLP/week7/airline_review_training_data.csv
test_data = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/NLP/week7/airline_review_test_data.csv")
test_data
test_data_clean = (
    # start with training data
    test_data
    # remove all the '\n' values with space
     .assign(clean_document= lambda x: [re.sub(r"&amp;","",text) for text in x.text])
    #Remove “n\”
    .assign(clean_document= lambda x: [re.sub(r"\n","",text) for text in x.clean_document] )
    #Remove mentions i.e. any alphanumeric starting with “@”
    .assign(clean_document= lambda x: [re.sub("@[A-Za-z0-9_]+","",text) for text in x.clean_document])
    #Remove all hashtags i.e. “#”
    .assign(clean_document= lambda x: [re.sub("#[A-Za-z0-9_]+","",text) for text in x.clean_document])
#     
#     .assign(clean_document= lambda x: [re.sub(r"#","",text) for text in x.clean_document])
#    Remove all links i.e. any alphanumeric starting with https or http
    .assign(clean_document= lambda x: [re.sub(r"http\S+","",text) for text in x.clean_document])   
)
test_data_clean

# use same vectorizer to transform the data
# transform(raw_documents)================>Transform documents to document-term matrix.
X_test = vectorizer.transform([i for i in test_data_clean['clean_document']])
df_tfid_sklearn_test = pd.DataFrame(X_test.toarray(),columns=vectorizer.get_feature_names())
X_test =df_tfid_sklearn_test.iloc[:, :-1] # aLL ROWS EXCEPT LAST
Y_test =df_tfid_sklearn_test.iloc[:, -1] # lAST row
Y_test = Y_test.astype(int)
test_data_clean['airline_sentiment']



/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


0       0
1       1
2       0
3       0
4       0
       ..
3458    1
3459    0
3460    0
3461    0
3462    0
Name: airline_sentiment, Length: 3463, dtype: int64

In [23]:
X_test

,about,after,again,airline,all,am,an,and,are,as,...,waiting,was,we,what,when,why,will,with,would,you
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.311435,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.425578,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.176932,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.324706,0.0,...,0.000000,0.0,0.318786,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.319869,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3458,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.222394,0.000000,0.0,...,0.000000,0.0,0.000000,0.363889,0.0,0.000000,0.0,0.000000,0.0,0.0
3459,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
3460,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.550656,0.0,0.000000,0.0,0.0
3461,0.0,0.0,0.0,0.0,0.0,0.0,0.306182,0.000000,0.000000,0.0,...,0.377147,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0


In [27]:
# Prediction and Classification Report
from sklearn.metrics import accuracy_score, classification_report

y_pred = classifier.predict(X_test)
y_pred_rf = clf.predict(X_test)
y_pred_svm =clf_SVM.predict(X_test)

# Classification metrics
def generate_classrpt(Y_test, y_pred,algo=''):
    print('======================================================')
    if algo!= "":
        print(" Clasification report for {algo}".format(algo=algo))
    classification_report_= classification_report(Y_test, y_pred)
    print('\n Accuracy: ', accuracy_score(Y_test, y_pred))
    print('\nClassification Report')
    print('======================================================')
    print('\n', classification_report_)
    return True

generate_classrpt(Y_test, y_pred,algo='MNB')
generate_classrpt(Y_test, y_pred_rf,algo='Random Forest')
generate_classrpt(Y_test, y_pred_svm,algo='SVM')




 Clasification report for MNB

 Accuracy:  0.9269419578400231

Classification Report

               precision    recall  f1-score   support

           0       1.00      0.93      0.96      3462
           1       0.00      0.00      0.00         1

    accuracy                           0.93      3463
   macro avg       0.50      0.46      0.48      3463
weighted avg       1.00      0.93      0.96      3463

 Clasification report for Random Forest

 Accuracy:  0.9997112330349408

Classification Report

               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3462
           1       0.00      0.00      0.00         1

    accuracy                           1.00      3463
   macro avg       0.50      0.50      0.50      3463
weighted avg       1.00      1.00      1.00      3463

 Clasification report for SVM

 Accuracy:  0.8830493791510251

Classification Report

               precision    recall  f1-score   support

           0       1

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


True

In [ ]:
from sklearn import model_selection
import joblib
# save the model to disk
filename = 'finalized_MNB_model.sav'
joblib.dump(classifier, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load('finalized_MNB_model.sav')
result = loaded_model.score(X_test, Y_test)
print(result)

0.9301183944556742


In [ ]:
from textblob import TextBlob
''' 1 is positive sentiment 0 is negative'''

# it accepts single text
# for a corpus, we need to run it in a loop

# Polarity score ranges from -1 to 1 
# -1 means very negative , 0 means nutral and +1 means positive sentiment

# classification using textblob 
test_data_clean['blob_sntmnt'] = test_data_clean['clean_document'].apply(lambda x: TextBlob(x).sentiment[0])
# blob sentiment is 0 if setiment value is negative and 1(postive) if the sentiment value is 0 or greater
test_data_clean['cln_blob_sntmnt'] =test_data_clean['blob_sntmnt'].apply(lambda x:0 if x<0  else 1)
generate_classrpt(test_data_clean['airline_sentiment'],test_data_clean['cln_blob_sntmnt'] ,algo='Text Blob')
test_data_clean

 Clasification report for Text Blob

 Accuracy:  0.46318221195495235

Classification Report

               precision    recall  f1-score   support

           0       0.97      0.33      0.50      2754
           1       0.27      0.96      0.42       709

    accuracy                           0.46      3463
   macro avg       0.62      0.65      0.46      3463
weighted avg       0.83      0.46      0.48      3463



,text,airline_sentiment,clean_document,blob_sntmnt,cln_blob_sntmnt
0,@AmericanAir @louprice13 @aacustomerservice. A...,0,. Apology not good enough. 4 planes with mec...,-0.175000,0
1,@united counter agents at RDU deserve a medal....,1,counter agents at RDU deserve a medal.,0.000000,1
2,@AmericanAir u Cancelled Flightled my flight f...,0,u Cancelled Flightled my flight for tomorrow ...,0.350000,1
3,@United Airlines Cancelled Flights OC FLL #fli...,0,Airlines Cancelled Flights OC FLL today. $70...,0.000000,1
4,@united I mean is there a real live person som...,0,I mean is there a real live person somewhere ...,-0.053977,0
...,...,...,...,...,...
3458,@SouthwestAir Whoa. Thanks and that's what I w...,1,Whoa. Thanks and that's what I wanted to hear...,0.183333,1
3459,@united UA647 heading back to gate bc improper...,0,UA647 heading back to gate bc improper load? ...,-0.150000,0
3460,@JetBlue why won't the site let me book ticket...,0,why won't the site let me book tickets for no...,0.000000,1
3461,@USAirways HELLO?! It's been FOUR HOURS since ...,0,HELLO?! It's been FOUR HOURS since plane land...,0.000000,1


In [ ]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


test_data_clean['vader_sntmnt'] = test_data_clean['clean_document'].apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x)['compound'])
test_data_clean['cln_vader_sntmnt'] =test_data_clean['vader_sntmnt'].apply(lambda x:0 if x<0  else 1)
generate_classrpt(test_data_clean['airline_sentiment'],test_data_clean['cln_vader_sntmnt'] ,algo='Vader')
test_data_clean



 Clasification report for Vader

 Accuracy:  0.5960150158821831

Classification Report

               precision    recall  f1-score   support

           0       0.98      0.50      0.66      2754
           1       0.33      0.97      0.50       709

    accuracy                           0.60      3463
   macro avg       0.66      0.73      0.58      3463
weighted avg       0.85      0.60      0.63      3463



,text,airline_sentiment,clean_document,blob_sntmnt,cln_blob_sntmnt,vader_sntmnt,cln_vader_sntmnt
0,@AmericanAir @louprice13 @aacustomerservice. A...,0,. Apology not good enough. 4 planes with mec...,-0.175000,0,-0.7274,0
1,@united counter agents at RDU deserve a medal....,1,counter agents at RDU deserve a medal.,0.000000,1,0.4767,1
2,@AmericanAir u Cancelled Flightled my flight f...,0,u Cancelled Flightled my flight for tomorrow ...,0.350000,1,-0.3018,0
3,@United Airlines Cancelled Flights OC FLL #fli...,0,Airlines Cancelled Flights OC FLL today. $70...,0.000000,1,-0.3527,0
4,@united I mean is there a real live person som...,0,I mean is there a real live person somewhere ...,-0.053977,0,0.3400,1
...,...,...,...,...,...,...,...
3458,@SouthwestAir Whoa. Thanks and that's what I w...,1,Whoa. Thanks and that's what I wanted to hear...,0.183333,1,0.7263,1
3459,@united UA647 heading back to gate bc improper...,0,UA647 heading back to gate bc improper load? ...,-0.150000,0,0.4939,1
3460,@JetBlue why won't the site let me book ticket...,0,why won't the site let me book tickets for no...,0.000000,1,0.0000,1
3461,@USAirways HELLO?! It's been FOUR HOURS since ...,0,HELLO?! It's been FOUR HOURS since plane land...,0.000000,1,-0.4184,0


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 4.9 MB/s 
     |████████████████████████████████| 596 kB 24.2 MB/s 
     |████████████████████████████████| 895 kB 8.7 MB/s 
     |████████████████████████████████| 67 kB 2.8 MB/s 
     |████████████████████████████████| 6.5 MB 21.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import pipeline


classifier = pipeline("sentiment-analysis")
test_data_clean = (
    test_data_clean
    #  classification using hugging face
    .assign(hugn_face = lambda x: x['clean_document'].apply(lambda s: classifier(s)))
# get the label and scroe from hugging face into different columns
    .assign(
         hgn_label = lambda x: x['hugn_face'].apply(lambda s: (s[0]['label'])),
         hgn_score = lambda x: x['hugn_face'].apply(lambda s: (s[0]['score']))
    )
    # if sentiment is negative mark as 0 other wise 1
    .assign(cln_hgn_sentiment = lambda x:x['hgn_label'].apply(lambda x:0 if x=='NEGATIVE'  else 1))
)


test_data_clean


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


,text,airline_sentiment,clean_document,blob_sntmnt,cln_blob_sntmnt,vader_sntmnt,cln_vader_sntmnt,hugn_face,hgn_label,hgn_score,cln_hgn_sentiment
0,@AmericanAir @louprice13 @aacustomerservice. A...,0,. Apology not good enough. 4 planes with mec...,-0.175000,0,-0.7274,0,"[{'label': 'NEGATIVE', 'score': 0.999204456806...",NEGATIVE,0.999204,0
1,@united counter agents at RDU deserve a medal....,1,counter agents at RDU deserve a medal.,0.000000,1,0.4767,1,"[{'label': 'POSITIVE', 'score': 0.969671964645...",POSITIVE,0.969672,1
2,@AmericanAir u Cancelled Flightled my flight f...,0,u Cancelled Flightled my flight for tomorrow ...,0.350000,1,-0.3018,0,"[{'label': 'NEGATIVE', 'score': 0.992760658264...",NEGATIVE,0.992761,0
3,@United Airlines Cancelled Flights OC FLL #fli...,0,Airlines Cancelled Flights OC FLL today. $70...,0.000000,1,-0.3527,0,"[{'label': 'NEGATIVE', 'score': 0.994259357452...",NEGATIVE,0.994259,0
4,@united I mean is there a real live person som...,0,I mean is there a real live person somewhere ...,-0.053977,0,0.3400,1,"[{'label': 'NEGATIVE', 'score': 0.997421860694...",NEGATIVE,0.997422,0
...,...,...,...,...,...,...,...,...,...,...,...
3458,@SouthwestAir Whoa. Thanks and that's what I w...,1,Whoa. Thanks and that's what I wanted to hear...,0.183333,1,0.7263,1,"[{'label': 'POSITIVE', 'score': 0.999566972255...",POSITIVE,0.999567,1
3459,@united UA647 heading back to gate bc improper...,0,UA647 heading back to gate bc improper load? ...,-0.150000,0,0.4939,1,"[{'label': 'NEGATIVE', 'score': 0.999549686908...",NEGATIVE,0.999550,0
3460,@JetBlue why won't the site let me book ticket...,0,why won't the site let me book tickets for no...,0.000000,1,0.0000,1,"[{'label': 'NEGATIVE', 'score': 0.997347354888...",NEGATIVE,0.997347,0
3461,@USAirways HELLO?! It's been FOUR HOURS since ...,0,HELLO?! It's been FOUR HOURS since plane land...,0.000000,1,-0.4184,0,"[{'label': 'NEGATIVE', 'score': 0.987911462783...",NEGATIVE,0.987911,0


In [ ]:
# Comparison of classification report with other algos
generate_classrpt(test_data_clean['airline_sentiment'],test_data_clean['cln_hgn_sentiment'] ,algo='Hugging Face')

 Clasification report for Hugging Face

 Accuracy:  0.893156222928097

Classification Report

               precision    recall  f1-score   support

           0       0.96      0.90      0.93      2754
           1       0.69      0.86      0.77       709

    accuracy                           0.89      3463
   macro avg       0.83      0.88      0.85      3463
weighted avg       0.91      0.89      0.90      3463



True

In [ ]:
generate_classrpt(test_data_clean['airline_sentiment'],test_data_clean['cln_vader_sntmnt'] ,algo='Vader')
generate_classrpt(test_data_clean['airline_sentiment'],test_data_clean['cln_blob_sntmnt'] ,algo='Text Blob')
generate_classrpt(Y_test, y_pred,algo='MNB')



 Clasification report for Vader

 Accuracy:  0.5960150158821831

Classification Report

               precision    recall  f1-score   support

           0       0.98      0.50      0.66      2754
           1       0.33      0.97      0.50       709

    accuracy                           0.60      3463
   macro avg       0.66      0.73      0.58      3463
weighted avg       0.85      0.60      0.63      3463

 Clasification report for Text Blob

 Accuracy:  0.46318221195495235

Classification Report

               precision    recall  f1-score   support

           0       0.97      0.33      0.50      2754
           1       0.27      0.96      0.42       709

    accuracy                           0.46      3463
   macro avg       0.62      0.65      0.46      3463
weighted avg       0.83      0.46      0.48      3463

 Clasification report for MNB

 Accuracy:  0.9301183944556742

Classification Report

               precision    recall  f1-score   support

           0       1.

True

In [ ]:
""" 
Reference:
https://medium.com/swlh/text-classification-using-tf-idf-7404e75565b8
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
https://datatofish.com/string-to-integer-dataframe/
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html
# Saving Model
https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/

"""
    